Extra Trees Model for Building Destruction Classification

In [1]:
#Import some basic packages
import numpy as np
import pandas as pd


In [2]:
#Import sklearn packages
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix


In [ ]:
#Let's import the data and instanciate a train/test split with the seed we fixed.

cal_fire_data = pd.read_csv('../data/cal_fire_data_cleaned.csv')
cal_fire_data.info()


In [ ]:

from sklearn.model_selection import train_test_split
X = cal_fire_data.drop(columns='Destroyed')
y = cal_fire_data['Destroyed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=216, shuffle=True, stratify=y)


We're trying an Extra Trees Classifier model.


In [ ]:
#Let's do the same but with Extra Trees model and compare.

categorical_cols = X.select_dtypes(include='object').columns.tolist()

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(['Destroyed'])

original_columns = cal_fire_data.columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('scaler', StandardScaler(), numeric_cols)
    ],
    remainder='passthrough'  
)

etpipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', ExtraTreesClassifier(n_estimators=100, random_state=216))
])

param_grid = {
    'classifier__criterion': ['gini'],
    'classifier__max_depth': [5, 10, 15, 20, None],
    'classifier__min_samples_split': [2, 4, 6, 8, 10, 12],
    'classifier__min_samples_leaf': [1, 2, 4, 8]
}

grid_search_et = GridSearchCV(etpipeline, param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search_et.fit(X_train, y_train)

print("best_params:", grid_search_et.best_params_)
y_etpred = grid_search_et.best_estimator_.predict(X_test)
print(classification_report(y_test, y_etpred))



Fitting 5 folds for each of 120 candidates, totalling 600 fits
best_params: {'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 8}
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      5115
           1       0.88      0.77      0.82      3443

    accuracy                           0.87      8558
   macro avg       0.87      0.85      0.86      8558
weighted avg       0.87      0.87      0.86      8558



In [ ]:
best_et = grid_search_et.best_estimator_
print('The confusion matrix is \n', confusion_matrix(y_test.values, y_etpred))
print('The accuracy score is', accuracy_score(y_test.values, y_etpred), '\n')
print('The precision score is', precision_score(y_test.values, y_etpred), '\n')
print('The recall score is', recall_score(y_test.values, y_etpred), '\n')
print('The f1 score is', f1_score(y_test.values, y_etpred), '\n')


The confusion matrix is 
 [[4759  356]
 [ 788 2655]]
The accuracy score is 0.8663239074550129 

The precision score is 0.8817668548654932 

The recall score is 0.7711298286378159 

The f1 score is 0.8227455841338704 



It's clear that the regular Random Forest Classifier offers much better performance, so we won't investigate this model further.

In [6]:
from IPython.display import display

# Use impurity-based feature importance (sum across all encodings for each feature)
importances = np.zeros(14)
importances[0] = np.sum(best_et['classifier'].feature_importances_[0:16])
importances[1] = np.sum(best_et['classifier'].feature_importances_[16:22])
importances[2] = np.sum(best_et['classifier'].feature_importances_[22:25])
importances[3] = np.sum(best_et['classifier'].feature_importances_[25:29])
importances[4] = np.sum(best_et['classifier'].feature_importances_[29:37])
importances[5] = np.sum(best_et['classifier'].feature_importances_[37:40])
importances[6] = np.sum(best_et['classifier'].feature_importances_[40:44])
importances[7] = np.sum(best_et['classifier'].feature_importances_[44:48])
importances[8] = np.sum(best_et['classifier'].feature_importances_[48:51])
importances[9] = np.sum(best_et['classifier'].feature_importances_[51:54])
importances[10:,] = best_et['classifier'].feature_importances_[54:58]

# Create a new dataframe for feature importances across original feature columns and display it
feature_df = pd.DataFrame({'feature': original_columns[:-1], 'importance': importances})
display(feature_df)


,feature,importance
0,* Structure Type,0.061844
1,* Roof Construction,0.043108
2,* Eaves,0.026809
3,* Vent Screen,0.039496
4,* Exterior Siding,0.440180
5,* Window Pane,0.021306
6,* Deck/Porch On Grade,0.021559
7,* Deck/Porch Elevated,0.019803
8,* Patio Cover/Carport Attached to Structure,0.018768
9,* Fence Attached to Structure,0.032648


For this model, the most important features were exterior siding and longitude.